In [2]:
from collections import deque, defaultdict
import re

In [3]:
input_spec = open('input.txt').readlines()
test_spec = '''Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.'''.splitlines()

In [127]:
class UTM(object):
    def __init__(self):
        self.tape = defaultdict(int)
        self.cursor = 0
        self.state = None
        self.iters = 0
        self.stop = None
        self.spec = defaultdict(dict)
        
    def step(self):
        s = self.state
        c = self.tape[self.cursor]
        self.tape[self.cursor] = self.spec[s][c]['write']
        self.cursor += self.spec[s][c]['move']
        self.state = self.spec[s][c]['new_state']
        self.iters += 1
        return self
    
    def diagnose(self):
        while self.iters < self.stop:
            self.step()
        return self
        
    def checksum(self):
        return sum(self.tape.values())
    
    
    def load(self, speclines):
        self.state = re.match(r'Begin in state ([A-Z]).', speclines[0]).group(1)
        self.stop = int(re.match(r'Perform a diagnostic checksum after ([0-9]+) steps.', speclines[1]).group(1))
        
        state = current = new = slot_move = new_state = None
        for l in speclines[2:]:
            #print('l: ', l)
            m = re.match(r'In state ([A-Z]):', l)
            if m:
                state = m.group(1)
                #print('state', state)
                
            m = re.match(r'  If the current value is ([01]):', l)
            if m:
                current = int(m.group(1))
                #print('current', current)
                
            m = re.match(r'    - Write the value ([01]).', l)
            if m:
                new = int(m.group(1))
                #print('new', new)
                
            m = re.match(r'    - Move one slot to the (left|right).', l)
            if m:
                slot_move = -1 if m.group(1) == 'left' else 1
                
            m = re.match(r'    - Continue with state ([A-Z]).', l)
            if m:
                new_state = m.group(1)
                
            if new_state is not None:
                self.spec[state][current] = {'write': new, 'move': slot_move, 'new_state': new_state}
                current = new = slot_move = new_state = None
                
        return self
    
    def dump(self):
        keys = self.tape.keys()
        if keys:
            left = min(keys)
            right = max(keys)
        else:
            left = right = 0
        curr = self.cursor
#         left = min([curr, left])
#         right = max([curr, right])
        tape = ''
        for i in range(left, right + 1):
            tape += f'{self.tape[i]}'
        n = int(tape, base=2)
        r = int(tape[::-1], base=2)
        s = (tape, self.iters, n, r, left, right, curr, self.state)
        return s
    
    def __str__(self):
        tape, iters, n, r, left, right, curr, state = self.dump()
        return f't="{tape:48}", i={iters}, n={n}, r={r}, min={left}, max={right}, current={curr}, state="{state}"'

    
    __repr__ = __str__

In [128]:
# %%timeit -n 1 -r 1
# test = UTM()
# test.load(test_spec)

# print(f'test (should be 3): {test.diagnose().checksum()}')

In [129]:
# %%timeit -n 1 -r 1
# part1 = UTM()
# part1.load(input_spec)
# print(f'part 1 answer: {part1.diagnose().checksum()}')

In [160]:
machine = UTM().load(input_spec)
evolution = []
vals = []
for i in range(31415):
    evolution.append(str(machine))
    vals.append(machine.dump())
    machine.step()

In [161]:
print("\n".join(evolution[:5]))

t="0                                               ", i=0, n=0, r=0, min=0, max=0, current=0, state="A"
t="1                                               ", i=1, n=1, r=1, min=0, max=0, current=1, state="B"
t="11                                              ", i=2, n=3, r=3, min=0, max=1, current=0, state="A"
t="01                                              ", i=3, n=1, r=2, min=0, max=1, current=-1, state="C"
t="101                                             ", i=4, n=5, r=5, min=-1, max=1, current=0, state="A"


In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [164]:
vals[-1]

('1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101010101010101010101010101010100010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010000001',
 31414,
 63657374260452690195888927762659156029123370060037422097968440910800644311493382823914113,
 32160235954499536192714718713494414326496164418227805163082934266380739743379086139981823,
 -280,
 14,
 -184,
 'B')

In [163]:
df = pd.DataFrame(vals, columns=['tape', 'iters', 'n', 'r', 'left', 'right', 'curr', 'state'])
df.head()

OverflowError: int too big to convert

AttributeError: module 'numpy' has no attribute 'int128'

In [152]:
f_at_left = df.curr == df.left
f_at_right = df.curr == df.left
f_state_A = df.state == 'A'
f_state_B = df.state == 'B'
f_state_C = df.state == 'C'
f_state_D = df.state == 'D'
f_state_E = df.state == 'E'
f_state_F = df.state == 'F'
f_all_ones = ~df.tape.str.contains('0')

In [149]:
df[f_all_ones].head()

,tape,iters,n,r,left,right,curr,state
1,1,1,1,1,0,0,1,B
2,11,2,3,3,0,1,0,A
5,111,5,7,7,-1,1,1,B
6,111,6,7,7,-1,1,2,D
7,1111,7,15,15,-1,2,3,A


In [153]:
df[f_state_F]

,tape,iters,n,r,left,right,curr,state
18,10001001,18,137,145,-3,4,-4,F
47,10001010101001,47,8873,9553,-6,7,-7,F
65,1010101000101001,65,43561,37973,-8,7,-9,F
101,101010101010100001,101,174753,136533,-10,7,-11,F
154,100010101010101010011,154,1135955,1660241,-13,7,-14,F
172,10101010001010101010011,172,5576019,6640725,-15,7,-16,F
208,1010101010101000101010011,208,22368595,26547541,-17,7,-18,F
262,101010101010101010100010011,262,89478419,105207125,-19,7,-20,F
351,1000101010101010101010101010101001,351,9305762473,10021590353,-22,11,-23,F
369,101010100010101010101010101010101001,369,45678766761,40086361173,-24,11,-25,F


In [151]:
df[f_all_ones]

,tape,iters,n,r,left,right,curr,state
1,1,1,1,1,0,0,1,B
2,11,2,3,3,0,1,0,A
5,111,5,7,7,-1,1,1,B
6,111,6,7,7,-1,1,2,D
7,1111,7,15,15,-1,2,3,A
8,11111,8,31,31,-1,3,4,B
9,111111,9,63,63,-1,4,3,A
28,111111111,28,511,511,-4,4,4,B
29,111111111,29,511,511,-4,4,5,D
30,1111111111,30,1023,1023,-4,5,6,A


In [157]:
df['checksum'] = df.tape.apply(lambda x: sum(map(int, list(x))))

In [159]:
df.head(20)

,tape,iters,n,r,left,right,curr,state,checksum
0,0,0,0,0,0,0,0,A,0
1,1,1,1,1,0,0,1,B,1
2,11,2,3,3,0,1,0,A,2
3,01,3,1,2,0,1,-1,C,1
4,101,4,5,5,-1,1,0,A,2
5,111,5,7,7,-1,1,1,B,3
6,111,6,7,7,-1,1,2,D,3
7,1111,7,15,15,-1,2,3,A,4
8,11111,8,31,31,-1,3,4,B,5
9,111111,9,63,63,-1,4,3,A,6
